In [5]:
# perform lemmatization (not recommended when using any pretrained embedding)
LEMMA = False
# size of the vocab for
VOCAB_SIZE = 20000
# for self attention
DATA_POINTS = 10
DIMENSIONS = 240
MAX_SEQUENCE_LENGTH = 256
# use gpu
USE_GPU = 0

In [6]:
!pip install tensorflow

You should consider upgrading via the '/Users/Robert1/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [7]:
!pip install keras

You should consider upgrading via the '/Users/Robert1/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 219856906614766031
xla_global_id: -1
]


In [11]:
# !python -m spacy download en_core_web_md

     |████████████████████████████████| 45.7 MB 2.2 MB/s            
You should consider upgrading via the '/Users/Robert1/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [12]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense, LSTM, Flatten, concatenate, Activation, RepeatVector, Permute
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
#from keras_self_attention import SeqSelfAttention
from random import randint
from numpy import array, argmax, asarray, zeros
import numpy as np 
import pandas as pd
import spacy
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textacy import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
nlp = spacy.load('en_core_web_md')

In [13]:
tf.config.list_physical_devices('GPU')

[]

In [14]:
df = pd.read_csv('complaints_processed.csv')
df.head()


,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


In [15]:
complaint_types = list(df['product'].value_counts().index)

In [16]:
NUM_CLASSES = len(complaint_types)

In [17]:
df['narrative'] = df['narrative'].apply(str)

In [18]:
preproc = preprocessing.make_pipeline(
    preprocessing.normalize.hyphenated_words,
    preprocessing.remove.accents,
)

In [19]:
set(stopwords.words('english'))
# removing some negative words from stopwords list
nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords.remove('below')
nltk_stopwords.remove("aren't")
nltk_stopwords.remove('couldn')
nltk_stopwords.remove("couldn't")
nltk_stopwords.remove("didn't")
nltk_stopwords = list(nltk_stopwords)
# add some abstract terms
nltk_stopwords.append('like')
nltk_stopwords.append('please')

nltk_stopwords = set(nltk_stopwords)

In [20]:
# removing all words with more than 15 digits
df['narrative'] = df['narrative'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)

/var/folders/5h/6l6vm3d54370g77lny2pkqc80000gp/T/ipykernel_15494/868481445.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['narrative'] = df['narrative'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)


In [21]:
# removing all characters that appear more than 2 times
df['narrative'] = df['narrative'].str.replace(r'(.)\1{3,}?', '', case=False)


/var/folders/5h/6l6vm3d54370g77lny2pkqc80000gp/T/ipykernel_15494/1200782744.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['narrative'] = df['narrative'].str.replace(r'(.)\1{3,}?', '', case=False)


In [22]:
labels = df['product']

2. model preparation
    1. tokenize
    2. integer encoding
    

In [23]:
# tokenize text
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(df['narrative'])

In [24]:
# integer encoding
from typing import List
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

# get the max length in terms of token length
max_length = get_max_token_length_per_doc(df['narrative'])


def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

from keras.preprocessing.sequence import pad_sequences
# integer encode the documents
encoded_docs = integer_encode_documents(df['narrative'], tokenizer)
# this is a list of lists, the numbers represent the index position of that word.
# for instance, 33 means the 33rd word in the vocabulary
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [29]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_labels = encoder.fit_transform(labels)
# print(transfomed_labels)

In [30]:
# %pip install skmultilearn

ERROR: Could not find a version that satisfies the requirement skmultilearn (from versions: none)
ERROR: No matching distribution found for skmultilearn
You should consider upgrading via the '/Users/Robert1/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
# we did not use stratifty due to multiclass complications
X_train, X_test, y_train, y_test = iterative_train_test_split(padded_docs, transfomed_labels, test_size=0.2)

3. Bert
- Modified after: https://colab.research.google.com/github/CyberZHG/keras-bert/blob/master/demo/tune/keras_bert_classification_tpu.ipynb

In [25]:
BATCH_SIZE = 128
EPOCHS = 5
LR = 1e-4

In [26]:
%pip install -q keras-bert keras-rectified-adam

You should consider upgrading via the '/Users/Robert1/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
/bin/bash: wget: command not found
unzip:  cannot find or open uncased_L-12_H-768_A-12.zip, uncased_L-12_H-768_A-12.zip.zip or uncased_L-12_H-768_A-12.zip.ZIP.


In [27]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [28]:
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=MAX_SEQUENCE_LENGTH,
)

FileNotFoundError: [Errno 2] No such file or directory: 'uncased_L-12_H-768_A-12/vocab.txt'

In [ ]:
from tensorflow.python import keras
from keras_radam import RAdam


inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(
    RAdam(lr=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [ ]:
model.fit(
    X_train,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)